# Steroids or Pentoxifylline for severe Alcoholic Hepatitis (STOPAH) trial


## Computation of Sobol indices and Shapley effects/values 


#### Implementation of Shapley values, Sobol indices and Shapley effects to prognostic scoring 

### MELD score

Model for End-Stage Liver Disease (MELD) score

\begin{equation}
\begin{array}{r}
\text { MELD }=(0.957 \times \ln (\text { creatinine }(\mathrm{mg} / \mathrm{dL})))+(0.378 \times \ln (\text { bilirubin }(\mathrm{mg} / \mathrm{dL})))+ \\
(1.120 \times \ln (\mathrm{INR}))+(0.643 \times \text { aetiology })
\end{array}
\end{equation}

A modification for large MELD Scores is 

\begin{equation}
\text { MELD-Na }=\left\{\begin{array}{ll}
\text { MELD }+1.32 \times(137-\text { sodium })-0.033 \times \text { MELD } \times(137-\text { sodium }), & \text { if MELD }>11 \\
\text { MELD }, & \text { otherwise }
\end{array}\right.
\end{equation}

### Lille Score 

\begin{equation}
\begin{array}{r}
\mathrm{R}=3.19-(0.101 \times \text { age }(\text { years }))+(0.147 \times \text { albumin }(\mathrm{g} / \mathrm{L}))+(0.0165 \times \Delta \text { bilirubin })- \\
(0.206 \times \text { renal insufficiency }(0 \text { or } 1))-(0.0065 \times \text { day } 0 \text { bilirubin }(\mu \mathrm{mol} / \mathrm{L}))- \\
(0.0096 \times \text { prothrombin time }(\text { seconds }))
\end{array}
\end{equation}

When is a patient considered for a liver transplant:

1. Lille Score > 0.45

2. MELT Score 

3. Chronic Liver Failure Consortium acute-on-chronic liver failure (CLIF-C ACLF) score of ≥ 70

As these scores are linear combinations of the values, the Shapley values are simply each term. 

Our aim is to extend the named scores by generalized versions via GAMS.

In [1]:
import os
import pandas as pd
import missingno as msno

from pygam import * 

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

from sklearn.metrics import auc, roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score 

from sklearn.utils.class_weight import compute_sample_weight



import numpy as np
import matplotlib.pyplot as plt
import sage 
import shap

import xgboost as xgb

/home/jlm217/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/home/jlm217/rds/rds-mrc-bsu-csoP2nj6Y6Y/mimah/stopah/stopah/data/stopah.csv')

In [3]:
# Original MELT Score

#Compute MELT Score 

df['MELT'] = 0.957 *np.log(df['Creatinine...Merged']) + 0.378 * np.log(df['Bilirubin.Merged']) + 1.120 * np.log(df['INR...Merged.clinical.and.calc'])

invalid value encountered in log


In [4]:
selected = ['D28_DTH','Prednisolone']

X_MELT = ['Creatinine...Merged','Bilirubin.Merged','INR...Merged.clinical.and.calc']

In [5]:
df = df[X_MELT+selected]

In [6]:
df = df.dropna()

df0 = df[df['Prednisolone']==0].drop(['Prednisolone'],axis=1)

df = df[df['Prednisolone']==1].drop(['Prednisolone'],axis=1)

X, y = df.drop('D28_DTH', axis=1), df[['D28_DTH']]

X0, y0 = df0.drop('D28_DTH', axis=1), df0[['D28_DTH']]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, random_state=1)

In [14]:
gam = LogisticGAM().fit(X_train.values,y_train.values)
gam0 = LogisticGAM().fit(X0_train.values,y0_train.values)

In [16]:
print('Untreated')
print()
print('XGB Score:'+str(gam0.score(X0_test, y0_test)))

y0_predict = gam0.predict_proba(X0_test)

auc_score0 = roc_auc_score(y0_test, y0_predict)
print('AUC Score:',(auc_score0)*100)

print()
print('Treated')
print()

print('XGB Score:'+str(gam.score(X_test, y_test)))

y_predict = gam.predict_proba(X_test)

auc_score = roc_auc_score(y_test, y_predict)
print('AUC Score:',(auc_score)*100)

Untreated

XGB Score:0.8432835820895522
AUC Score: 69.84577922077922

Treated

XGB Score:0.8872180451127819
AUC Score: 77.43961352657006


In [17]:
gam.summary()
gam0.summary()

LogisticGAM                                                                                               
=============================================== ==========================================================
Distribution:                      BinomialDist Effective DoF:                                     14.4067
Link Function:                        LogitLink Log Likelihood:                                  -142.7142
Number of Samples:                          399 AIC:                                              314.2419
                                                AICc:                                             315.5632
                                                UBRE:                                               2.8165
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                   0.1181
Feature Function                  Lam

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

